In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
pip install easyocr torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install apache-airflow-providers-celery
!pip install internetarchive
!pip install pandas
!pip install pika
!pip install pip-chill
!pip install pyarrow
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.0/426.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import sys
import os

# Add the directory containing archive_utils.py to the Python path
sys.path.append('/kaggle/input/archive-utils')

# Import the module
from archive_utils import get_identifiers_for_language, dowload_identifier_data, getItemMetaData

# Now you can use the functions from archive_utils
'''language = 'eng'  # Example language code
identifiers = get_identifiers_for_language(language)
download_identifier_data(identifiers)
metadata = getItemMetaData(identifiers[0])

print(metadata)'''


"language = 'eng'  # Example language code\nidentifiers = get_identifiers_for_language(language)\ndownload_identifier_data(identifiers)\nmetadata = getItemMetaData(identifiers[0])\n\nprint(metadata)"

In [ ]:
from archive_utils import get_identifiers_for_language, dowload_identifier_data, getItemMetaData
import os
import concurrent.futures
import pandas as pd
from tqdm import tqdm
import argparse


In [ ]:
languages = [
    # top resource languages
    'English',
    'Assamese',
    'Bengali',
    'Gujarati',
    'Hindi',
    'Kannada',
    'Malayalam',
    'Marathi',
    'Oriya',
    'Punjabi',
    'Tamil',
    'Telugu',

    # bottom resource languages
    'Bodo',
    'Dogri',
    'Konkani', 
    'Kashmiri',
    'Maithili',
    'Manipuri',
    'Nepali',
    'Sanskrit',
    'Santali',
    'Sindhi',
    'Urdu',
]

languages_code = [
    # top resource codes
    'eng',
    'asm',
    'ben',
    'guj',
    'hin',
    'kan',
    'mal',
    'mar',
    'ori',
    'pan',
    'tam',
    'tel',

    # bottom resource codes
    'brx',
    'doi',
    'kok', 
    'kas',
    'mai',
    'mni',
    'nep',
    'san',
    'sat',
    'snd', 
    'urd',
]


In [ ]:
def downloadIdentifiers(languages, languages_code, destdir):
    identifiers_list = []
    for language, language_code in zip(languages, languages_code):
        print(f"Curating Identifiers for {language} ({language_code})")
        identifiers = get_identifiers_for_language(language=language, language_code=language_code)
        for identifier in identifiers:
            identifiers_list.append([identifier, language])

    identifierDataframe = pd.DataFrame(data=identifiers_list, columns=["identifier", "language"])
    identifierDataframe.to_csv(f"{destdir}/identifiers.csv", index=False)
    print("Saved Identifiers to CSV Format")
    return f"{destdir}/identifiers.csv"


In [ ]:
def downloadHelper(arguments):
    dowload_identifier_data(
        identifier=arguments[0],
        language=arguments[1],
        destination_folder=arguments[2],
        pdf_only=arguments[3]
    )


In [ ]:
import sys

# Check if running in Jupyter notebook
IN_JUPYTER = 'ipykernel' in sys.argv[0]

if IN_JUPYTER:
    # If in Jupyter, simulate argument parsing
    args = {
        'languageList': ['English', 'Hindi'],  # Example list of languages
        'pdf_only': True,
        'id_only': False
    }
else:
    # If running from command line or other environment, use argparse
    import argparse
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--languages", action='store', dest='languageList', type=str, nargs='*', help="List of languages to download")
    parser.add_argument("--pdf_only", type=bool, default=True, help="To download only associated PDF Files")
    parser.add_argument("--id_only", type=bool, default=False, help="To download only identifiers file")
    
    args = parser.parse_args()

# Continue with your script using `args` dictionary
# Example usage
if isinstance(args, dict):  # Handle Jupyter notebook simulated args
    languageList = args.get('languageList', [])
    pdf_only = args.get('pdf_only', True)
    id_only = args.get('id_only', False)
else:  # Handle argparse Namespace
    languageList = args.languageList
    pdf_only = args.pdf_only
    id_only = args.id_only

print("Languages:", languageList)
print("PDF Only:", pdf_only)
print("ID Only:", id_only)

# Example function calls based on args
# downloadIdentifiers(languageList, ...)
# metaExecutor = concurrent.futures.ProcessPoolExecutor()
# ...



Languages: ['English', 'Hindi']
PDF Only: True
ID Only: False


In [ ]:
import sys
import os
import concurrent.futures
import pandas as pd
from tqdm import tqdm

# Simulate argument parsing if running in Jupyter notebook
IN_JUPYTER = 'ipykernel' in sys.argv[0]

if IN_JUPYTER:
    # Simulated arguments for Jupyter
    args = {
        'languageList': ['English', 'Hindi'],
        'pdf_only': True,
        'id_only': False
    }
else:
    # Actual argument parsing for command-line
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--languages", action='store', dest='languageList', type=str, nargs='*', help="List of languages to download")
    parser.add_argument("--pdf_only", type=bool, default=True, help="To download only associated PDF Files")
    parser.add_argument("--id_only", type=bool, default=False, help="To download only identifiers file")

    args = parser.parse_args()

# Define language dictionary
languages = [
    'English', 'Assamese', 'Bengali', 'Gujarati', 'Hindi', 'Kannada', 'Malayalam',
    'Marathi', 'Oriya', 'Punjabi', 'Tamil', 'Telugu', 'Bodo', 'Dogri', 'Konkani',
    'Kashmiri', 'Maithili', 'Manipuri', 'Nepali', 'Sanskrit', 'Santali', 'Sindhi', 'Urdu'
]
languages_code = [
    'eng', 'asm', 'ben', 'guj', 'hin', 'kan', 'mal', 'mar', 'ori', 'pan', 'tam',
    'tel', 'brx', 'doi', 'kok', 'kas', 'mai', 'mni', 'nep', 'san', 'sat', 'snd', 'urd'
]
language_dict = dict(zip(languages, languages_code))

# Function definitions
def downloadIdentifiers(languages_list, languages_codes, dest_dir):
    identifiers_list = []
    for language, code in zip(languages_list, languages_codes):
        print(f"Curating Identifiers for {language} ({code})")
        identifiers = get_identifiers_for_language(language=language, language_code=code)
        for identifier in identifiers:
            identifiers_list.append([identifier, language])

    identifier_df = pd.DataFrame(data=identifiers_list, columns=["identifier", "language"])
    identifiers_path = os.path.join(dest_dir, "identifiers.csv")
    identifier_df.to_csv(identifiers_path, index=False)
    print(f"Saved Identifiers to CSV Format: {identifiers_path}")
    return identifiers_path

def downloadHelper(arguments):
    download_identifier_data(identifier=arguments[0], language=arguments[1], destination_folder=arguments[2], pdf_only=arguments[3])

if __name__ == "__main__":
    languages_to_download_codes = [language_dict[x] for x in args['languageList']]
    data_path = "../../data/"
    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    if args['languageList']:
        identifiers_path = downloadIdentifiers(args['languageList'], languages_to_download_codes, data_path)
    else:
        identifiers_path = downloadIdentifiers(languages, languages_code, data_path)
    
    identifiers_df = pd.read_csv(identifiers_path)
    identifiers_with_language = [(row["identifier"], row["language"], data_path, args['pdf_only']) for idx, row in identifiers_df.iterrows()]
    
    meta_executor = concurrent.futures.ProcessPoolExecutor()
    print("Fetching Metadata for Identifiers...")
    identifiers = [x[0] for x in identifiers_with_language]
    results = list(tqdm(meta_executor.map(getItemMetaData, identifiers), total=len(identifiers)))
    
    result_df = pd.DataFrame(
        data=results,
        columns=[
            "identifier", "title", "ppi", "subject", "ocr", "description", 
            "date", "ialanguage", "uploader", "mediatype", "totalDataSize", "pdfDataSize"
        ],
    )
    
    meta_executor.shutdown()
    
    result_df.to_csv(f"{data_path}/identifiersMetaData.csv", index=False)
    print(f"Identifiers Metadata saved to {data_path}/identifiersMetaData.csv")
    
    if not args['id_only']:
        download_executor = concurrent.futures.ProcessPoolExecutor()
        print("Downloading Associated Files with Identifiers and Languages...")
        download_size = sum(result_df["pdfDataSize"]) if args['pdf_only'] else sum(result_df["totalDataSize"])
        print(f"Download Size: {download_size / (1024 * 1024)} MB")
        
        download_results = list(tqdm(download_executor.map(downloadHelper, identifiers_with_language), total=len(identifiers_with_language)))
        download_executor.shutdown()
        
        print("Files Downloaded from archive.org for associated Identifiers and Languages.")
    else:
        print("Download Complete")


Curating Identifiers for English (eng)


KeyboardInterrupt: 

In [ ]:
#original code
'''from archive_utils import get_identifiers_for_language,dowload_identifier_data,getItemMetaData
import os
import concurrent
import concurrent.futures
import pandas as pd
from tqdm import tqdm
import argparse

languages = [
    # top resource languages
    'English',
    'Assamese',
    'Bengali',
    'Gujarati',
    'Hindi',
    'Kannada',
    'Malayalam',
    'Marathi',
    'Oriya',
    'Punjabi',
    'Tamil',
    'Telugu',

    # bottom resource languages
    'Bodo',
    'Dogri',
    'Konkani', 
    'Kashmiri',
    'Maithili',
    'Manipuri',
    'Nepali',
    'Sanskrit',
    'Santali',
    'Sindhi',
    'Urdu',
]


languages_code = [
    # top resource codes
    'eng',
    'asm',
    'ben',
    'guj',
    'hin',
    'kan',
    'mal',
    'mar',
    'ori',
    'pan',
    'tam',
    'tel',

    # bottom resource codes
    'brx',
    'doi',
    'kok', 
    'kas',
    'mai',
    'mni',
    'nep',
    'san',
    'sat',
    'snd', 
    'urd',
]

language_dict = dict(zip(languages, languages_code))

def downloadIdentifiers(languages,languages_code,destdir):
    identifiers_list = []
    for language,language_code in list(zip(languages,languages_code)):
        print(f"Curating Identifiers for {language} ({language_code})")
        identifiers = get_identifiers_for_language(language=language,language_code=language_code)
        for identifier in identifiers:
            identifiers_list.append([identifier,language])

    identifierDataframe = pd.DataFrame(data=identifiers_list,columns=["identifier","language"])
    identifierDataframe.to_csv(f"{destdir}/identifiers.csv",index=False)
    print("Saved Identifiers to CSV Format")
    return f"{destdir}/identifiers.csv"

def downloadHelper(arguments):
    dowload_identifier_data(identifier=arguments[0],language=arguments[1],destination_folder=arguments[2],pdf_only=arguments[3])

parser = argparse.ArgumentParser()
parser.add_argument("--languages",action='store', dest='languageList',type=str, nargs='*',help="List of languages to download")
parser.add_argument("--pdf_only",type=bool,default=True,help="To download only associated PDF Files")
parser.add_argument("--id_only",type=bool,default=False,help="To download only identifiers file")

if __name__=="__main__":
    args = parser.parse_args()
    languagesToDownloadCodes = [language_dict[x] for x in args.languageList]
    data_path = "../../data/"
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    if args.languageList:
        identifiersPath = downloadIdentifiers(args.languageList,languagesToDownloadCodes,data_path)
    else:
        identifiersPath = downloadIdentifiers(languages,languages_code,data_path)
    identifiersDf = pd.read_csv(identifiersPath)
    identifiersWithLanguage = []
    for idx,row in identifiersDf.iterrows():
        identifiersWithLanguage.append((row["identifier"],row["language"],data_path,True if args.pdf_only else False))

    metaExecutor = concurrent.futures.ProcessPoolExecutor()
    print("Fetching Metadata for Identifiers...")
    identifiers = [x[0] for x in identifiersWithLanguage]
    results = list(
        tqdm(metaExecutor.map(getItemMetaData, identifiers), total=len(identifiers))
    )

    resultDf = pd.DataFrame(
        data=results,
        columns=[
            "identifier",
            "title",
            "ppi",
            "subject",
            "ocr",
            "description",
            "date",
            "ialanguage",
            "uploader",
            "mediatype",
            "totalDataSize",
            "pdfDataSize"
        ],
    )

    resultDf.to_csv(f"{data_path}/identifiersMetaData.csv")
    print(f"Identifiers Metadata saved to {data_path}/identifiersMetaData.csv")
    metaExecutor.shutdown()

    if not args.id_only:
        downloadExecutor = concurrent.futures.ProcessPoolExecutor()
        print("Downloading Associated Files with Identifiers and Languages...")
        downloadSize = sum(resultDf["pdfDataSize"]) if args.pdf_only else sum(resultDf["totalDataSize"])
        print(f"Download Size : {downloadSize/(1024*1024)} MB")
        downloadResults = list(
            tqdm(downloadExecutor.map(downloadHelper, identifiersWithLanguage), total=len(identifiersWithLanguage))
        )
        downloadExecutor.shutdown()
        print("Files Downloaded from archive.org for associated Identifiers and Languages.")
    else:
        print("Download Complete")'''


usage: ipykernel_launcher.py [-h] [--languages [LANGUAGELIST ...]]
                             [--pdf_only PDF_ONLY] [--id_only ID_ONLY]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-ba0523ca-b2c3-44c6-afe3-3190a7cbcdfa.json


SystemExit: 2